# IEEE Quantum Week Challenge: Transpilation and optimization using QAOA

**Welcome to the IEEE Quantum week Challenge on optimizing QAOA transpilation!** In this 2-hour challenge, you'll explore the process of optimizing quantum circuits using Qiskit SDK's transpiler and Qiskit Transpiler service on a Quantum Approximate Optimization Algorithm (QAOA) cost function as the target workload.

Your goal is to develop a custom transpiler workflow that can efficiently map QAOA circuits to a given quantum architecture, minimizing circuit depth and gate count while maintaining fidelity.

This notebook is divided into two parts: **Beginner** and **Advanced**, each with specific learning objectives, coding challenges, and a grading mechanism to assess performance. Let's get started!


# Table of Contents

**Beginner Track**
* [**Brief overview: Introduction to QAOA and Problem setting**](#Brief-Overview:-Introduction-to-the-Quantum-Approximate-Optimization-Algorithm-(QAOA))
* [**Basic Transpilation with Preset Pass Manager**](#2.-Basic-Transpilation-with-Preset-Pass-Manager)
    * [*Exercise 1*](#exercise_1)
    * [**Customize Transpilation Settings**](#Step-2:-Customize-Transpilation-Settings)
        * [*Exercise 2*](#exercise2)
    * [**Transpiling Multiple Times for Best Results**](#3.-Tip:-Transpiling-Multiple-Times-for-Best-Results)
        * [*Exercise 3*](#exercise3)
* [**AI Assisted Transpilation**](#3.-AI-Assisted-Transpilation)
    * [*Exercise 4*](#exercise4)
* [**Diving Deeper into Transpilation Using Custom Pass Managers**](#3.Diving-Deeper-into-Transpilation-Using-Custom-Pass-Managers)
* [**Final Challenge: Putting It All Together**](#Applying-High-Impact-Passes-Using-Custom-and-Staged-Pass-Managers)
    * [*Exercise 5*](#exercise5)
 
**Advanced Track**
* [**Advanced Track: Deep Dive into Custom Transpilation Strategies**](#Advanced-Track:-Deep-Dive-into-Custom-Transpilation-Strategies)
* [**Example: Custom optimal pass for QAOA**](#6.-Example:-Custom-optimal-pass-for-QAOA)
    * [*Exercise 6: Grand Finale*](#Exercise-6:-Grand-Finale)

In [ ]:
# Imports
from colorama import Fore, Style
from grader import grade_ex1, grade_ex2, grade_ex3, grade_ex4, grade_ex5, grade_ex6, score_func

# Brief Overview: Introduction to the Quantum Approximate Optimization Algorithm (QAOA)

## Quick Intro: What is QAOA?

The Quantum Approximate Optimization Algorithm (QAOA) is a quantum algorithm designed to tackle combinatorial optimization problems. These are problems where the goal is to find the optimal solution from a set of finite possible outcomes which are often encountered in tasks like routing, scheduling, or resource allocation. QAOA operates by applying quantum operations that approximate the optimal solution, making it a promising approach for potentially solving these problems efficiently on near-term quantum computers.

### Key Components of QAOA

QAOA works by encoding the problem into a quantum circuit that evolves through a series of unitary operations. The algorithm iteratively adjusts these operations to find the optimal solution. The two main components of QAOA are:

1. **Cost Function Hamiltonian (HC):** This is the problem-specific part of the circuit that encodes the objective function of the optimization problem. The goal of the QAOA is to find the parameters that minimize the expectation value of the cost function, driving the quantum system towards the optimal solution.

2. **Mixer Hamiltonian (HM):** This component of the circuit drives the exploration of the solution space. It typically uses single-qubit rotations that evolve the quantum state away from an initial, easily prepared state.

In this notebook, we will focus on optimizing the **cost function circuit** of QAOA. The cost function circuit is crucial because it directly relates to the problem you want to solve. Specifically, the circuit is constructed to reflect the objective function of your optimization problem.



### Why Optimize the Cost Function Circuit?

Optimizing the cost function circuit is crucial because it directly impacts the performance and accuracy of QAOA. The circuit’s depth and the number of gates can significantly affect the algorithm's ability to find the optimal solution, especially on noisy quantum hardware. By applying optimization techniques during the transpilation process, we can reduce the circuit's complexity, making it more suitable for execution on real quantum devices.

In this notebook, we will use Qiskit SDK's transpiler to optimize the cost function circuit of QAOA, starting with a basic implementation and gradually incorporating more advanced transpilation techniques and customizing our workflow.

Now, let's move on to implementing and optimizing the cost function circuit of QAOA using Qiskit SDK's Transpiler!

## 1. Problem setting - Example: Max-Cut Problem

As a concrete example, we'll start our workflow with a graph that represents a hypothetical max-cut problem, where the goal is to divide this graph into two sets such that the number of edges between the sets is maximized. The cost function Hamiltonian for our hypothetical Max-Cut problem is formulated as:

$$ H_C = \sum_{(i,j) \in E} \frac{1}{2} (1 - Z_i Z_j) $$

Here, $Z_i$ and $Z_j$ are Pauli-Z operators acting on qubits $i$ and $j$, and $E$ represents the set of edges in the graph. The cost function circuit for the Max-Cut problem thus involves a series of entangling gates (e.g., $RZZ$ gates defined as: $\text{RZZ}(\gamma) = e^{-i \gamma Z_i Z_j / 2}$  where $\gamma$ is a parameter to be optimized) that correspond to the edges in the graph. 


In [ ]:
from networkx import barabasi_albert_graph, draw

graph = barabasi_albert_graph(n=50, m=7, seed=42)

In [ ]:
draw(graph, with_labels=True)

The graph above corresponds to the following operators:

In [ ]:
from utils import build_max_cut_paulis
from qiskit.quantum_info import SparsePauliOp

local_correlators = build_max_cut_paulis(graph)
cost_operator = SparsePauliOp.from_list(local_correlators)
print(cost_operator)

## Constructing the Cost Function Circuit for a Simple Graph

Let’s construct the cost function circuit for this example. This operator maps to a 50 qubit QAOA circuit:

In [ ]:
num_qubits = cost_operator.num_qubits
print(num_qubits)

Now lets build QAOA circuit/ansatz. We will use the `QAOAAnsatz` fucntion in Qiskit to define it for us.

In [ ]:
from qiskit import QuantumCircuit
from qiskit.circuit.library import QAOAAnsatz
from qiskit.circuit import ParameterVector

# We are taking advantage of the QAOAAnsatz class to build the cost layer, 
# note that we are giving it dummy initial state and mixer circuits.

dummy_initial_state = QuantumCircuit(num_qubits)  # the real initial state is defined later
dummy_mixer_operator = QuantumCircuit(num_qubits)  # the real mixer is defined later

# Use off-the-shelf qiskit QAOAAnsatz
cost_layer = QAOAAnsatz(
    cost_operator,
    reps=1,
    initial_state=dummy_initial_state,
    mixer_operator=dummy_mixer_operator,
    name="QAOA cost block",
)

cost_layer.draw("mpl", fold=-1, scale=0.5)

## 2. Basic Transpilation with Preset Pass Manager

### Why Transpilation?

In quantum computing, a quantum circuit is a sequence of quantum gates applied to qubits to perform a computation. However, the circuit you initially design isn't always optimal for execution on a real quantum computer. This is where **transpilation** comes in.

**Transpilation** is the process of transforming a quantum circuit into an equivalent one that is optimized for a specific quantum device. This optimization process involves several steps, such as reducing the circuit depth (the number of gate layers), minimizing the number of gates, and mapping the logical qubits in your circuit to the physical qubits on the quantum hardware.

When you design a quantum circuit the abstract form that doesn't consider the limitations of real quantum hardware. To run this circuit on an actual quantum device, it must be transformed to match the topology (connectivity) and gate set of the device. Transpilation accomplishes this by:

1. **Matching the Topology:** Ensuring that the logical qubits in your circuit are mapped to the physical qubits on the device in a way that respects the hardware's connectivity constraints.
2. **Optimizing Instructions:** Rewriting the circuit's instructions to use only the gates supported by the device (Instruction Set Architecture) and minimizing the circuit's depth and gate count to reduce the effects of noise.

### Transpilation Stages

The process of transpilation in Qiskit is divided into several key stages, each designed to address different aspects of circuit optimization:

1. **`init`:** This stage involves preparing the circuit by converting any custom instructions into single- and two-qubit gates, which are standard types of gates that the QPU can execute. This step also validates the circuit's instructions.

2. **`layout`:** During this stage, the virtual qubits in your circuit are mapped to the physical qubits on the QPU. This mapping is crucial because it determines how the qubits interact with each other based on the device's connectivity.

3. **`routing`:** After applying the layout, the circuit may need additional gates (such as SWAP gates) to ensure that qubits can interact according to the device's coupling map. This stage injects these gates to maintain the circuit's logical structure while adhering to the hardware's connectivity.

4. **`translation`:** In this stage, the circuit's gates are translated into the specific set of instructions (basis gates) that the QPU can execute. This ensures that the circuit is compatible with the device's Instruction Set Architecture (ISA).

5. **`optimization`:** This stage involves applying various optimization passes to reduce the circuit's depth and gate count. The goal is to find more efficient ways to represent the circuit that minimize the potential for errors and noise during execution.

6. **`scheduling`:** The final stage is scheduling, which organizes the execution of the circuit's instructions to account for idle times and other hardware constraints. This stage is particularly important for optimizing circuits on near term devices.

For more details on the specifics of each of these stages, **check out this documentation link on [IBM Quantum Documentation](https://docs.quantum.ibm.com/guides/transpiler-stages)**

Let’s now select a target IBM Quantum backend from the Qiskit Runtime service to optimize our circuits. The backend target we are selecting is `ibm_torrino`

In [ ]:
from qiskit_ibm_runtime import QiskitRuntimeService

# QiskitRuntimeService.save_account(
#     channel="ibm_quantum",
#     token="",
#     set_as_default=True,
#     # Use `overwrite=True` if you're updating your token.
#     overwrite=True,
# )
 
# Load saved credentials
service = QiskitRuntimeService(channel="ibm_quantum")

In [ ]:
# Select backend
backend = service.backend("ibm_torino")

In [ ]:
from qiskit.visualization import plot_gate_map, plot_error_map
plot_error_map(backend)

### Using Preset Pass Managers

Qiskit provides a convenient way to perform transpilation through **preset pass managers** using [`generate_preset_pass_manager`](https://docs.quantum.ibm.com/guides/defaults-and-configuration-options). These are predefined sets of transpilation passes that optimize the circuit to varying degrees depending on the selected **optimization level**:

- **Level 0:** *[No Optimization]* - Basic translation with trivial qubit mapping and no optimizations, primarily for hardware characterization.
- **Level 1:** *[Light Optimization]* - Reduces gate count and simplifies the circuit with minimal compile time, using basic layout and gate optimization.
- **Level 2:** *[Medium Optimization]* - Applies more sophisticated heuristic-based optimizations to further reduce circuit complexity.
- **Level 3:** *[High Optimization]* - Performs the most extensive optimizations, including advanced gate resynthesis and in-depth layout adjustments.

Each optimization level offers a different balance between the quality of the optimized circuit and the time it takes to transpile. Higher levels generally produce better-optimized circuits but require more time to complete.

In this section, you will apply a preset pass manager to the QAOA cost function circuit designed to solve the Max-Cut problem. Initially, the circuit will be in its raw form, containing standard quantum gates like Hadamard and CNOT. By applying different levels of optimization, you will observe how the circuit is transformed into a more hardware-efficient version that uses gates supported by the chosen backend, such as RZ, X, SX, and ECR gates.

In [ ]:
# Pre-transpilation circuit
print(cost_layer.decompose().count_ops())

<a id="exercise_1"></a>
<div class="alert alert-block alert-success">
    
<b> Exercise 1: Explore Transpilation Levels</b> 

Now that you understand the basics of transpilation and the role of preset pass managers, it’s time to put this knowledge into practice. <b>Try transpiling the QAOA circuit you created earlier using `generate_preset_pass_manager` and explore different `optimization_levels`.</b> Observe how the circuit changes and consider which level of optimization provides the best balance between transpilation time and circuit efficiency for your specific problem.
</div> 

In [ ]:
# Import necessary libraries
import time
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

### Write your code below here ###

pm_ex1 = 

### Don't change any code past this line ###

###############  Measure the start time of the transpilation process
t0 = time.time()

# Run the pass manager to transpile the QAOA cost layer circuit
naively_transpiled_qaoa = pm_ex1.run(cost_layer)

# Measure the end time of the transpilation process
t1 = time.time()

# Print the total transpilation time
print(f"Transpilation time: {t1 - t0:.2f} (s)")

In [ ]:
# Extract information from the transpiled circuit
circuit = naively_transpiled_qaoa

# Output various metrics about the transpiled circuit
print(f'Count Ops: {circuit.count_ops()}')
print(f'Total circuit depth: {circuit.depth()}')
print(f'Number of 2-qubit gates: {circuit.size(lambda x: x.operation.num_qubits == 2)}')

# Output the score and depth specifically for 2-qubit gates, in color
print(Fore.MAGENTA + Style.BRIGHT + f'2-qubit gate depth: {circuit.depth(lambda x: x.operation.num_qubits == 2)}')
print(Fore.GREEN + f'Score: {score_func(circuit)}')

<a id="scoring_func"></a>
<div class="alert alert-block alert-info">
    
⚠️ **About the scoring function** 

The scoring function evaluates your transpiled circuit based on its depth and gate usage, focusing on optimizing two-qubit gates, which are more resource-intensive on quantum hardware. It first checks if the circuit only uses the backend's basis gates and conforms to its qubit connectivity. If either check fails, a high penalty score is applied. Otherwise, the score is calculated by summing the depth of two-qubit gates and a reduced weight of single-qubit gates as follows:

$$score = n_{\text{2Q gate depth}} + n_{\text{1Q gate depth}}/10$$

**The goal is to minimize the score. Lower scores indicate more efficient circuits and higher circuit fidelity, so strive for optimization!** </div>

In [ ]:
# Run this cell to grade your answer
grade_ex1(pm_ex1, backend)

<div class="alert alert-block alert-warning">
    
⚠️ **Please note** 

While transpiling a circuit multiple times can lead to better optimization outcomes, it's important to be mindful of scalability issues. Transpiling multiple times can significantly increase the total runtime, especially for large circuits. This method is more appropriate for smaller circuits or when higher levels of optimization are essential. Additionally, each transpilation pass involves computational overhead, and for larger circuits or when scaling to many circuits, performing multiple trials could potentially result in workflow being inefficient and unscalable.
</div>

### Customize Transpilation Settings

Next, consider customizing some of the parameters available in the `generate_preset_pass_manager` function to see how they affect the transpilation process. You can adjust settings related to the circuit's `init`, `layout`, `routing`, `translation`, `optimization`, and `scheduling` stages.

Explore changing parameters pertaining to the transpilation stages and observe how these changes impact circuit metrics:

- **Init Stage:** Customize the synthesis algorithms with hls_config, select a different initialization plugin using `init_method`, or adjust the unitary synthesis method with `unitary_synthesis_method` and `unitary_synthesis_plugin_config`.
- **Layout Stage:** Modify the `layout_method` to change how the initial layout of qubits is selected. Options include `trivial`, `dense`, `noise_adaptive`, or `sabre`.
- **Routing Stage:** Adjust the `routing_method` to influence how qubits are swapped during the circuit execution. Options include `basic`, `lookahead`, `stochastic`, `sabre`, or `none`.
- **Translation Stage:** Experiment with the translation_method to determine how the circuit is converted to the backend’s native gate set, choosing between `translator` or `synthesis`.
- **Optimization Stage:** Experiment with the `approximation_degree` to trade off accuracy for gate reduction. You can set it to a value between `0.0` (maximal approximation) and `1.0` (no approximation), or `None` to automatically match the error rate.
- **Scheduling Stage:** Control the timing of gate execution with the scheduling_method, selecting either `as_soon_as_possible` (ASAP) or `as_late_as_possible` (ALAP)

You can learn more about these options here in the documentation [here](https://docs.quantum.ibm.com/guides/defaults-and-configuration-options)

**Debugging tip:** When customizing transpilation settings, it can be extremely helpful to print out the list of passes that will be applied. This allows you to verify that your custom passes are being included in the correct order and that no unwanted default passes are being added. You can do this by adding a callback function as below:

In [ ]:
def callback_func(pass_, dag, time, property_set, count):
    print(f"PASS {count}: {pass_.name()} in {time}")
    print(f"ops: {dag.count_ops()}")
    print(f"dep: {dag.depth()}")
    print(f"========")

Examining the pass list helps you understand exactly how your circuit is being transpiled and can be invaluable for troubleshooting unexpected results or optimizing the transpilation process. If you notice passes that shouldn't be there or passes missing that you expected, you can adjust your [`generate_preset_pass_manager`](https://docs.quantum.ibm.com/api/qiskit/0.42/qiskit.transpiler.preset_passmanagers.generate_preset_pass_manager) configuration accordingly.

Experiment with modifying parameters associated with the transpilation stages to observe their impact on your circuit metrics:

<a id="exercise2"></a>
<div class="alert alert-block alert-success">
    
<b> Exercise 2: Explore Transpilation options</b> 

In this exercise, you'll dive deeper into the transpilation process by customizing key parameters using the [`generate_preset_pass_manager`](https://docs.quantum.ibm.com/api/qiskit/0.42/qiskit.transpiler.preset_passmanagers.generate_preset_pass_manager) function. Experiment with different stages of transpilation, such as `init`, `layout`, `routing`, `translation`, `optimization`,`scheduling` as well as explore the different attirbutes of the `generate_preset_pass_manager`. By modifying these parameters, you can observe how they affect the performance and characteristics of your quantum circuit. Aim to get a lower gate depth (Total or 2 qubit gate depth) than the default transpilation options</div> 

In [ ]:
# Feel free to play around with these options

### Write your code below here ###

pm_ex2 = generate_preset_pass_manager(
    optimization_level=None,
    backend=backend,
    instruction_durations=None,
    initial_layout=None,
    layout_method=None,
    routing_method=None,
    translation_method=None,
    scheduling_method=None,
    approximation_degree=None,
    seed_transpiler=None,
    unitary_synthesis_method=None,
    unitary_synthesis_plugin_config=None,
    hls_config=None,
    init_method=None,
    optimization_method=None,
)

### Don't change any code past this line ###


In [ ]:
import time
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

###############  Measure the start time of the transpilation process
t0 = time.time()

# Run the pass manager to transpile the QAOA cost layer circuit
naively_custom_transpiled_qaoa = pm_ex2.run(cost_layer, callback=callback_func)

# Measure the end time of the transpilation process
t1 = time.time()

# Print transpilation time
print(f"Transpilation time: {t1 - t0} (s)")

In [ ]:
# Extract information from the transpiled circuit
circuit = naively_custom_transpiled_qaoa

# Output various metrics about the transpiled circuit
print(f'Count Ops: {circuit.count_ops()}')
print(f'Total circuit depth: {circuit.depth()}')
print(f'Number of 2-qubit gates: {circuit.size(lambda x: x.operation.num_qubits == 2)}')

# Output the score and depth specifically for 2-qubit gates, in color
print(Fore.MAGENTA + Style.BRIGHT + f'2-qubit gate depth: {circuit.depth(lambda x: x.operation.num_qubits == 2)}')
print(Fore.GREEN + f'Score: {score_func(circuit)}')

In [ ]:
# Run this cell to grade your answer
grade_ex2(pm_ex2, backend)


### 3. Tip: Transpiling Multiple Times for Best Results
When working with quantum circuits, the process of transpilation involves many stochastic passes, which means the results can vary each time you transpile the same circuit. In practice, running the transpiler multiple times may lead to better results in terms of circuit depth, gate count, and overall optimization. This is because some of the randomization involved in the transpilation process might occasionally yield a more efficient circuit layout or routing. One effective method to achieve more predictable outcomes is to use the `seed_transpiler` option. By varying a seed, you can control the randomization of the transpilation process, making it easier to reproduce and compare results.

Perform the process multiple times and keep the best outcome based on your chosen metric (e.g., minimal gate count, execution time). This can be achieved by running a loop over several transpilation attempts, each with a fixed seed, and selecting the most optimized circuit.

#### Try transpiling the circuit multiple times to observe variations in the results and choose the most optimal one.

Here’s an example where we transpile a circuit multiple times, each time storing the result and comparing the transpiled circuits based on gate count:



<div class="alert alert-block alert-danger">
    
⚠️ **Disclaimer on using this strategy:** 

While transpiling a circuit multiple times can lead to better optimization outcomes, it's important to be mindful of scalability issues. Transpiling multiple times can significantly increase the total runtime, especially for large circuits. This method is more appropriate for smaller circuits or when higher levels of optimization are essential. Additionally, each transpilation pass involves computational overhead, and for larger circuits or when scaling to many circuits, performing multiple trials could potentially result in workflow being inefficient and unscalable.
</div>

<a id="exercise2"></a>
<div class="alert alert-block alert-success">
    
<b> Exercise 3: Transpile Multiple times</b> 
It's time to explore the impact of running the transpiler multiple times. For this exercise, transpile the same circuit multiple times, and aim to achieve an optimized version by selecting the best result from these attempts
</div>

In [ ]:
import time
from colorama import Fore, Style  # Ensure colorama is imported for colored output

# Initialize the depth to a large number for comparison
depth = 2000

# Record the starting time of the transpilation process
t0 = time.time()

### Write your code below here ###

# Hint: Loop over a range of seeds to find the best transpilation result


seed_selected = 

### Don't change any code past this line ###

###############  Measure the start time of the transpilation process
t1 = time.time()

# Output the selected seed that resulted in the best transpilation result
print(f'Seed selected: {seed_selected}')

# Use the circuit with the best depth
circuit = circuit_routed

# Print transpilation time
print(f"Transpilation time: {t1 - t0} (s)")

# Output various metrics about the transpiled circuit
print(f'Count Ops: {circuit.count_ops()}')
print(f'Total circuit depth: {circuit.depth()}')
print(f'Number of 2-qubit gates: {circuit.size(lambda x: x.operation.num_qubits == 2)}')

# Output the score and depth specifically for 2-qubit gates, in color
print(Fore.MAGENTA + Style.BRIGHT + f'2-qubit gate depth: {circuit.depth(lambda x: x.operation.num_qubits == 2)}')
print(Fore.GREEN + f'Score: {score_func(circuit)}')

In [ ]:
# Run this cell to grade your answer
pm_ex3 = generate_preset_pass_manager(backend=backend, optimization_level=3, seed_transpiler=seed_selected)
grade_ex3(pm_ex3, backend)

## 4. AI-Assisted Transpilation

### Leveraging AI in Transpilation

For this section, we are providing you with beta access to the Qiskit Transpiler Service to experiment and explore the cutting-edge capabilities of AI-powered transpiler passes in Qiskit. With the recent beta release of the Qiskit Transpiler Service, you can now leverage AI and IBM Cloud&reg; resources to optimize your quantum circuits more efficiently.

In this step, you'll transpile and optimize your circuit using the Qiskit Transpiler Service. We will be using the Qiskit Transpiler Service for this section. The Qiskit Transpiler Service provides both heuristic and AI-powered transpilation capabilities on the cloud. We can leverage running transpilation tasks to benefit from IBM Quantum™ cloud resources and advanced AI-powered transpiler passes. **For more information, refer to the Qiskit Transpiler Service documentation [here](https://docs.quantum.ibm.com/transpile/qiskit-transpiler-service)**


<div class="alert alert-block alert-danger">

Please note, **these features are in beta**. Have fun experimenting and pushing the boundaries of quantum development and let us know if you have any feedback on how to make this tool better!

</div>

<a id="exercise4"></a>
<div class="alert alert-block alert-success">
    
<b> Exercise 4: Exploring AI-powered transpilation</b> 

In this exercise, you'll use the Qiskit Transpiler service with AI-powered transpilation. First, ensure you have the latest version of the Qiskit Transpiler Service installed. Continue using the `ibm_sherbrooke` backend and create a new transpiler configuration with `optimization_level` 3 and the AI flag enabled (use `ai=True`). Transpile your quantum circuit using this new configuration and compare the results (gate count, depth) with your previous transpilation. 

**For an additional challenge**, experiment embedding AI-Powered passes like `AIRouting`, `AICliffordSynthesis`, `AILinearFunctionSynthesis` and `AIPermutationSynthesis` using PassManagers. Documentation for the same can be found [here.](https://docs.quantum.ibm.com/guides/ai-transpiler-passes)
</div> 


In [ ]:
# The transpiler service uses token from your QiskitRuntimeService.

# QiskitRuntimeService.save_account(
#     channel="ibm_quantum",
#     token="",
#     set_as_default=True,
#     # Use `overwrite=True` if you're updating your token.
#     overwrite=True,
# )
 
# Load saved credentials
service = QiskitRuntimeService(channel="ibm_quantum")

In [ ]:
# Setting up the Transpiler service
from qiskit_ibm_transpiler import TranspilerService

### Write your code below here ###

ts_ex4 = 

### Don't change any code past this line ###


###############  Measure the start time of the transpilation process
t0_ai_transpiled = time.time()

# Run the pass manager to transpile the QAOA cost layer circuit
circuit_ai_transpiled = ts_ex4.run(cost_layer)

# Measure the end time of the transpilation process
t1_ai_transpiled  = time.time()
time_with_ai = t1_ai_transpiled - t0_ai_transpiled

You can also use AI-powered transpiler passes as a drop-in custom pass in a staged pass manager. Please check the documentation [here](https://docs.quantum.ibm.com/guides/ai-transpiler-passes) for more details.

In [ ]:
# Extract information from the transpiled circuit
circuit = circuit_ai_transpiled

# Print transpilation time
print(f"Transpilation time: {time_with_ai} (s)")

# Output various metrics about the transpiled circuit
print(f'Count Ops: {circuit.count_ops()}')
print(f'Total circuit depth: {circuit.depth()}')
print(f'Number of 2-qubit gates: {circuit.size(lambda x: x.operation.num_qubits == 2)}')

# Output the score and depth specifically for 2-qubit gates, in color
print(Fore.MAGENTA + Style.BRIGHT + f'2-qubit gate depth: {circuit.depth(lambda x: x.operation.num_qubits == 2)}')
print(Fore.GREEN + f'Score: {score_func(circuit)}')

In [ ]:
# Run this cell to grade your answer
grade_ex4(ts_ex4, backend)

## 5. Diving Deeper into Transpilation Using Custom Pass Managers

### Introduction to Transpilation Strategies: Cheat Sheet

In quantum computing, optimizing circuits is crucial for improving the performance and accuracy of quantum algorithms on real hardware. While preset pass managers in Qiskit provide a solid foundation for general optimization, there are cases where more targeted, deeper strategies are needed to achieve better results. Customizing transpilation through specific strategies allows you to refine circuit performance by reducing gate count, minimizing circuit depth, and optimizing qubit mapping for your specific hardware.

Let us dive deeper into these strategies and leverage Qiskit SDK's `PassManager`, to experiment with various techniques to improve the efficiency of your circuits, enabling them to run more effectively on real quantum devices.

### General Strategies Cheat Sheet

| **Strategy**                             | **When to Use This Strategy?**                                                        | **Why?**                                                                                                                                        | **How Much Improvement?**                                                                                       | **Implementation**                                                                                                                        |
|------------------------------------------|--------------------------------------------------------------------------------------|------------------------------------------------------------------------------------------------------------------------------------------------|-----------------------------------------------------------------------------------------------------------------|-----------------------------------------------------------------------------------------------------------------------------------------|
| **Try transpiling multiple times**       | Always (May not be needed in most cases now. Use `SabreLayout` and increase `trial_count` argument instead.) | Many transpilation passes are stochastic. Running the transpiler multiple times may result in better outcomes.                                                     | Transpile multiple times in a loop and keep the best result.                                                   | Utilize a loop or a genetic algorithm to mutate and evolve a better transpiler workflow.                                                                       |
| **Dropping the final Clifford (operator backpropagation)** | When measuring an observable                                                  | Can potentially reduce the circuit depth with minimal overhead cost. | May help optimize depth without much extra complexity. | Implement operator backpropagation by rearranging gate sequences.                                                                                                                                         |
| **Use approximation degree**               | For deep circuits                                                                    | Deep circuits often accumulate noise, so trading off accuracy with fewer gates/errors may improve results.         | Can reduce the depth by optimizing gates based on the error rates of qubit gates. | Use `approximation_degree='none'` or set it based on hardware error rates.                                                                                                                                         |
| **Apply dynamical decoupling**              | For devices with high crosstalk errors (e.g., Eagle); may not be effective in all devices            | Dynamical decoupling can suppress crosstalk errors between qubits.                                                                                               | Potential improvements for specific hardware architectures. | Use Qiskit Runtime’s dynamical decoupling capabilities or create a custom decoupling pass.                                                                                                                              |
| **Numerical approximate methods**        | Large networks of CNOTs or Clifford gates                                             | Useful when dealing with many gates on relatively few qubits.                                                                                           | Can simplify circuits with large gate counts. | Explore numerical approximation methods for gate simplification.                                                                                                                                         |
| **Use SabrePreLayout**                   | When trading size for depth, or with dynamic circuit support.                                                                 | Helps reduce overall circuit depth by optimizing layout before routing.                                                                                               | Useful when depth is a more critical factor than size. | Use Sabre layout or explore Qiskit’s `SabrePreLayout` for more dynamic control.                                                            |
| **Leverage Pauli synthesis tools**       | For circuits with large sequences of multi-qubit Pauli rotations or Trotter decomposition problems. | Optimizing Pauli rotations can significantly reduce circuit depth, especially in hardware with limited connectivity. | May result in substantial depth reduction in circuits with many Pauli rotations. | Use Pauli synthesis tools in Qiskit or Rustiq for Trotter circuits.                                                                                             |

---




### Applying Transpilation Strategies Using Custom and Staged Pass Managers

In this section, you’ll learn how to integrate these optimization strategies into your custom pass manager to improve your quantum circuit. In Qiskit SDK, Custom `PassManagers` allow fine-grained control over the transpilation process by enabling users to define a specific sequence of optimization and transformation passes. These passes can target specific goals such as reducing circuit depth, improving fidelity, or adapting to the hardware topology.  By applying a combination of these methods, you can see how each strategy contributes to the overall circuit efficiency, making it more suitable for execution on IBM Quantum backends.

The `StagedPassManager` organizes the transpilation workflow into distinct stages, each stage being a `PassManager` focusing on different optimization goals such as initial layout selection, gate reduction, or final qubit mapping. This staged approach allows for incremental improvements and conditional passes that apply only when necessary, offering a structured way to fine-tune the balance between execution speed and circuit fidelity. Together, Custom `PassManagers` and `StagedPassManager` enable deeper exploration of transpilation strategies, giving developers full control over the optimization process to create circuits best suited for specific hardware constraints.

### Custom Pass Managers

As mentioned above, a **`PassManager`** allows you to apply a sequence of optimization passes to a quantum circuit, transforming it for more efficient execution on quantum hardware. By creating a custom Pass Manager, you can tailor the optimization to your circuit's specific needs, achieving deeper optimization compared to the preset configurations. Lets look at how we can create a custom `PassManager` and apply it to a workflow.

#### Example: Creating a Custom Pass Manager

To create a custom Pass Manager, you can combine various strategies together to form a transformation. Here is an example using `Collect2qBlocks`, `ConsolidateBlocks`, and `UnitarySynthesis` to optimize the circuit. You can refer to the documentation links [here](https://docs.quantum.ibm.com/api/qiskit/qiskit.transpiler.passes.Collect2qBlocks) to know more about their transformation.


In [ ]:
from qiskit.transpiler import PassManager
from qiskit.transpiler.passes import Collect2qBlocks, ConsolidateBlocks, UnitarySynthesis

# Show example 
basis_gates = ["rx", "ry", "rxx"]
custom_pass_manager = PassManager([
   Collect2qBlocks(),
   ConsolidateBlocks(basis_gates=basis_gates),
   UnitarySynthesis(basis_gates),
])


You can then apply this custom Pass Manager to a circuit using its run method.

In [ ]:
###############  Measure the start time of the transpilation process
t0 = time.time()

# Run the pass manager to transpile the QAOA cost layer circuit
transpiled_circuit = custom_pass_manager.run(cost_layer)

# Measure the end time of the transpilation process
t1 = time.time()

# Print transpilation time
print(f"Transpilation time: {t1-t0} (s)")

transpiled_circuit.decompose(reps=2).draw(output="mpl",idle_wires=False, fold=-1,scale=0.5)

### Staged Pass Managers
A **Staged Pass Manager** allows you to organize multiple `PassManagers` into distinct stages of transpilation such as  initialization, layout, routing, translation, optimization, and scheduling. This provides more flexibility in how the transpilation process is structured.

You can create a Staged Pass Manager by defining stages and assigning Pass Managers to each stage:

In [ ]:
from qiskit.transpiler import PassManager, StagedPassManager
from qiskit.transpiler.passes import UnitarySynthesis, Unroll3qOrMore
# Preset pass manager is the staged pass manager and flow from there. Use the preset passmanager and swap a stage
init_pm = PassManager([UnitarySynthesis(basis_gates=["rx", "ry", "rxx"], min_qubits=3), Unroll3qOrMore()])
translate_pm = PassManager([Collect2qBlocks(), ConsolidateBlocks(basis_gates=["rx", "ry", "rxx"])])

staged_pass_manager = StagedPassManager(stages=["init", "translation"], init=init_pm, translation=translate_pm)

In [ ]:
###############  Measure the start time of the transpilation process
t0 = time.time()

# Run the pass manager to transpile the QAOA cost layer circuit
transpiled_circuit = staged_pass_manager.run(cost_layer)

# Measure the end time of the transpilation process
t1 = time.time()

# Print the total transpilation time
print(f"transpilation time: {t1 - t0:.2f} (s)")
transpiled_circuit.decompose().draw(output="mpl",idle_wires=False, fold=-1,scale=0.5)

You can also modify stages of Qiskit preset passmanagers and modify the workflow:

In [ ]:
circuit = transpiled_circuit

# Output various metrics about the transpiled circuit
print(f'Count Ops: {circuit.count_ops()}')
print(f'Total circuit depth: {circuit.depth()}')
print(f'Number of 2-qubit gates: {circuit.size(lambda x: x.operation.num_qubits == 2)}')

# Output the score and depth specifically for 2-qubit gates, in color
print(Fore.MAGENTA + Style.BRIGHT + f'2-qubit gate depth: {circuit.depth(lambda x: x.operation.num_qubits == 2)}')
print(Fore.GREEN + f'Score: {score_func(circuit)}')

**Now that you have learned how to create a custom `PassManager`, `StagedPassManager` and AI-assisted passes, let us put it to test on our cost function circuit**



<a id="exercise5"></a>
<div class="alert alert-block alert-success">
    
<b> Exercise 5: Exploring custom pass managers</b> 

In the next exercise, use all of the above techniques to get to a better score for optimizing the QAOA Cost function circuit. This exercise will help you appreciate the advanced optimization techniques that the Qiskit SDK's transpiler has that enhance the performance of quantum circuits. It will also provide a foundation for experimenting with more complex customizations in the Advanced Track section of this notebook.

**Challenge:**
1. Use a preset `PassManager` or a custom `PassManager` to optimize the QAOA cost function transpilation as much as possible. You can experiment to design your own `StagedPassManager` to integrate your custom `PassManager` workflow.
2. Try to achieve the best possible performance by experimenting with different combinations of passes.
3. Feel free to make use of AI-Powered transpiler passes or external plugins from the [Qiskit Ecosystem](https://www.ibm.com/quantum/ecosystem?tag=Transpiler+plugin) to acheive the same
4. The score as assessed by the grader needs to be less than **400** for the `cost_layer` circuit provided

**Hint:**  
Remember that sometimes adding too many passes can lead to diminishing returns or even degrade performance. Aim for a balance! Once you've finalized your pass manager, submit your optimized circuit for grading.

**Let’s apply these strategies to optimize your QAOA circuit!**
</div> 






#### Useful Resources for Transpiler Passes

- General documentation for passes: [Qiskit Transpiler Passes Documentation](https://docs.quantum.ibm.com/api/qiskit/transpiler_passes)

#### Specific Pass Categories:
- **Layout Passes**: [Layout Passes](https://github.com/Qiskit/qiskit/tree/main/qiskit/transpiler/passes/layout)
- **Routing Passes**: [Routing Passes](https://github.com/Qiskit/qiskit/tree/main/qiskit/transpiler/passes/routing)
- **Translation Passes**: [Translation Passes](https://github.com/Qiskit/qiskit/tree/main/qiskit/transpiler/passes/synthesis)
- **Optimization Passes**: [Optimization Passes](https://github.com/Qiskit/qiskit/tree/main/qiskit/transpiler/passes/optimization)


In [ ]:
# Import necessary libraries and transpiler passes
from qiskit.circuit import SessionEquivalenceLibrary
from qiskit.transpiler import PassManager, StagedPassManager
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.transpiler.preset_passmanagers.common import generate_unroll_3q, generate_embed_passmanager
from qiskit.transpiler.passes.layout import (
    CSPLayout, DenseLayout, SabreLayout, VF2Layout, TrivialLayout
)
from qiskit.transpiler.passes.routing import (
    BasicSwap, LookaheadSwap, SabreSwap, StochasticSwap, StarPreRouting
)
from qiskit.transpiler.passes.basis import BasisTranslator
from qiskit.transpiler.passes.optimization import (
    CommutativeCancellation, ConsolidateBlocks, Collect2qBlocks, Collect1qRuns, Optimize1qGates, Optimize1qGatesDecomposition, Optimize1qGatesSimpleCommutation
)
from qiskit.transpiler.passes.synthesis import UnitarySynthesis, HighLevelSynthesis


# Set up coupling map for transpilation stages
cm = backend.coupling_map

### Write your code below here ###

pm_ex5 = 





### Don't change any code past this line ###

###############  Measure the start time of the transpilation process
t0 = time.time()

# Run the pass manager to transpile the QAOA cost layer circuit
pm_aprox_transpiled_qaoa = pm_ex5.run(cost_layer, callback=callback_func)

# Measure the end time of the transpilation process
t1 = time.time()

# Print transpilation time
print(f"Transpilation time: {t1 - t0:.2f} seconds")

In [ ]:
circuit = pm_aprox_transpiled_qaoa

# Output various metrics about the transpiled circuit
print(f'Count Ops: {circuit.count_ops()}')
print(f'Total circuit depth: {circuit.depth()}')
print(f'Number of 2-qubit gates: {circuit.size(lambda x: x.operation.num_qubits == 2)}')

# Output the score and depth specifically for 2-qubit gates, in color
print(Fore.MAGENTA + Style.BRIGHT + f'2-qubit gate depth: {circuit.depth(lambda x: x.operation.num_qubits == 2)}')
print(Fore.GREEN + f'Score: {score_func(circuit)}')

In [ ]:
# Run this cell to grade your answer
grade_ex5(pm_ex5, backend)

<a id="congratulations"></a>
<div class="alert alert-block alert-success">
    
<b> 🎉 **Congratulations!** 🎉 on Completing Part 1! </b>



You've successfully completed the first part of the challenge, where you've gained a solid understanding of Qiskit’s transpilation process and learned how to optimize quantum circuits using preset Pass Managers. Your ability to navigate through the basics and apply these optimizations is a significant achievement. 

But why stop here? There’s a whole new level of customization and control waiting for you in the **Advanced Track**.



</div> 


## Advanced Track: Deep Dive into Custom Transpilation Strategies

<div class="alert alert-block alert-warning">

<b> Welcome to the Advanced section: Ready for the Next Challenge? </b> 

In the Advanced Track, you'll take your skills to the next level by diving deep into custom transpilation strategies. Here, you'll have the opportunity to write your own transpiler passes, experiment with advanced Pass Managers, and fine-tune your circuits like never before.

This next stage is all about pushing boundaries and seeing how far you can optimize your quantum circuits. It’s where the real challenge begins, and where you’ll truly master the art of quantum circuit optimization.

**Note: This section does not require you to do the beginner section and can be attempted standalone.**

Are you ready to take on the Advanced Challenge? Let’s continue this journey and see how far you can go!

</div>


### Introduction

For those looking to push the limits of quantum circuit optimization, the Advanced Track offers a deeper exploration into custom transpilation strategies. This track is designed for participants who want to experiment with more sophisticated techniques, leveraging the full flexibility of Qiskit’s Transpilation capablities.

### Challenge Objectives

- **Optimize Beyond Defaults:** Move beyond preset configurations to develop and fine-tune custom Pass Managers that implement high-impact optimizations tailored to your circuit's unique requirements.
- **Write Custom Transpiler Passes:** Attempt to create your own transpiler passes to address specific optimization challenges using your own approaches.
- **Experiment with Staged Transpilation:** Utilize Staged Pass Managers to structure the transpilation process into distinct phases and precisly control over each stage of circuit transformation.


Benchmark your optimizations and test and iterate on your custom strategies, comparing them against standard presets to demonstrate your works effectiveness against the grader.


To get you inspired, here is an example of a custom transpilation pass strategy implemented by our IBM researchers:

## 6. Example: Custom optimal pass for QAOA
This section shows how applying domain-specific transpilation strategies allows to reduce circuit depth and 2-qubit gate count on qaoa circuits. 

*This section is from a notebook authored by Elena Peña Tapia, IBM Quantum. You can find the notebook here: [https://github.com/qiskit-community/qopt-best-practices/blob/main/how_tos/how_to_apply_optimal_qaoa_transpilation.ipynb](https://github.com/qiskit-community/qopt-best-practices/blob/main/how_tos/how_to_apply_optimal_qaoa_transpilation.ipynb)*

References:

[1] Sack, S. H., & Egger, D. J. (2023). Large-scale quantum approximate optimization on non-planar graphs with machine learning noise mitigation. arXiv preprint arXiv:2307.14427.

[2] Weidenfeller, J., Valor, L. C., Gacon, J., Tornow, C., Bello, L., Woerner, S., & Egger, D. J. (2022). Scaling of the quantum approximate optimization algorithm on superconducting qubit based hardware. Quantum, 6, 870. 

<div class="alert alert-block alert-warning">

<b> **Note:** </b> The problem setting is the example below is for $n=10$ and $m=6$

</div>

In [ ]:
# Setup problem
from qiskit import QuantumCircuit
from networkx import barabasi_albert_graph, draw
num_qubits = 10
graph = barabasi_albert_graph(n=num_qubits, m=6, seed=42)

# Build operators
from utils import build_max_cut_paulis
from qiskit.quantum_info import SparsePauliOp

local_correlators = build_max_cut_paulis(graph)
cost_operator = SparsePauliOp.from_list(local_correlators)
print(cost_operator)

# Build cost_layer
# We are taking advantage of the QAOAAnsatz class to build the cost layer, 
# note that we are giving it dummy initial state and mixer circuits.
from qiskit import QuantumCircuit
from qiskit.circuit.library import QAOAAnsatz
from qiskit.circuit import ParameterVector

dummy_initial_state = QuantumCircuit(num_qubits)  # the real initial state is defined later
dummy_mixer_operator = QuantumCircuit(num_qubits)  # the real mixer is defined later

cost_layer = QAOAAnsatz(
    cost_operator,
    reps=1,
    initial_state=dummy_initial_state,
    mixer_operator=dummy_mixer_operator,
    name="QAOA cost block",
)

In [ ]:
from qiskit_ibm_runtime import QiskitRuntimeService

# QiskitRuntimeService.save_account(
#     channel="ibm_quantum",
#     token="",
#     set_as_default=True,
#     # Use `overwrite=True` if you're updating your token.
#     overwrite=True,
# )
 
# Load saved credentials
service = QiskitRuntimeService(channel="ibm_quantum")

# Select backend
backend = service.backend("ibm_torino")

### Design pass manager for cost layer transpilation

The cost layer transpilation stage will apply the swap strategies from [1,2] to optimally route the cost layer circuit into the connectivity of our backend. This collection of passes will insert swap gates to fit the circuit to the chip connectivity in a way that will allow to maximally cancel CNOT gates and reduce the total circuit depth.

Note that we are routing the circuit into a `line`, which proves to be optimal in most QAOA cases. To make sure that the line fits into our chip, we use the `BackendEvaluator` utility before committing to the swap strategy:

In [ ]:
from utils import BackendEvaluator

# The backend evaluator finds the line of qubits with the best fidelity to map the circuit to
path_finder = BackendEvaluator(backend)
path, fidelity, num_subsets = path_finder.evaluate(num_qubits)
print(path, fidelity)

The collection of passes defined below will be embedded into the "pre_init" stage of our `StagedPassManager`.

In [ ]:
from qiskit.transpiler import PassManager
from qiskit.transpiler.passes import (
    BasisTranslator,
    UnrollCustomDefinitions,
    CommutativeCancellation,
    Decompose,
    CXCancellation,
    HighLevelSynthesis,
    InverseCancellation
)

from qiskit.transpiler.passes.routing.commuting_2q_gate_routing import (
    SwapStrategy,
    FindCommutingPauliEvolutions,
    Commuting2qGateRouter,
)
from qiskit.circuit.library.standard_gates.equivalence_library import _sel
from qiskit.circuit.library import CXGate

In [ ]:
# 1. choose swap strategy (in this case -> line)
swap_strategy = SwapStrategy.from_line([i for i in range(num_qubits)])
edge_coloring = {(idx, idx + 1): (idx + 1) % 2 for idx in range(num_qubits)}

# 2. define pass manager for cost layer
pre_init = PassManager(
            [HighLevelSynthesis(basis_gates=['PauliEvolution']),
             FindCommutingPauliEvolutions(),
             Commuting2qGateRouter(
                    swap_strategy,
                    edge_coloring,
                ),
             HighLevelSynthesis(basis_gates=["x", "cx", "sx", "rz", "id"]),
             InverseCancellation(gates_to_cancel=[CXGate()]),
            ]
)

We can run this pass manager independently to confirm that it performs the desired transformations:

In [ ]:
# Before:
print(cost_layer.decompose(reps=4).count_ops())
cost_layer.decompose(reps=4).draw("mpl", fold=-1)

In [ ]:
tmp = pre_init.run(cost_layer)

In [ ]:
# After:
print(tmp.count_ops())
tmp.draw('mpl', fold=-1)

In [ ]:
# Mixer operator = rx rotations
betas = ParameterVector("β", qaoa_layers)
mixer_operator = QuantumCircuit(num_qubits)
mixer_operator.rx(-2*betas[0], range(num_qubits))

In [ ]:
from qiskit.transpiler.basepasses import TransformationPass
from qiskit.converters import circuit_to_dag, dag_to_circuit

class QAOAPass(TransformationPass):

    def __init__(self, num_layers, num_qubits, init_state = None, mixer_layer = None):

        super().__init__()
        self.num_layers = num_layers
        self.num_qubits = num_qubits
        
        if init_state is None:
            # Add default initial state -> equal superposition
            self.init_state = QuantumCircuit(num_qubits)
            self.init_state.h(range(num_qubits))
        else: 
            self.init_state = init_state
        
        if mixer_layer is None:
            # Define default mixer layer
            self.mixer_layer = QuantumCircuit(num_qubits)
            self.mixer_layer.rx(-2*betas[0], range(num_qubits))
        else:
            self.mixer_layer = mixer_layer

    def run(self, cost_layer_dag):

        cost_layer = dag_to_circuit(cost_layer_dag)
        qaoa_circuit = QuantumCircuit(self.num_qubits, self.num_qubits)
        # Re-parametrize the circuit
        gammas = ParameterVector("γ", self.num_layers)
        betas = ParameterVector("β", self.num_layers)

        # Add initial state
        qaoa_circuit.compose(self.init_state, inplace = True)

        # iterate over number of qaoa layers
        # and alternate cost/reversed cost and mixer
        for layer in range(self.num_layers): 
        
            bind_dict = {cost_layer.parameters[0]: gammas[layer]}
            bound_cost_layer = cost_layer.assign_parameters(bind_dict)
            
            bind_dict = {self.mixer_layer.parameters[0]: betas[layer]}
            bound_mixer_layer = self.mixer_layer.assign_parameters(bind_dict)
        
            if layer % 2 == 0:
                # even layer -> append cost
                qaoa_circuit.compose(bound_cost_layer, range(self.num_qubits), inplace=True)
            else:
                # odd layer -> append reversed cost
                qaoa_circuit.compose(bound_cost_layer.reverse_ops(), range(self.num_qubits), inplace=True)
        
            # the mixer layer is not reversed
            qaoa_circuit.compose(bound_mixer_layer, range(self.num_qubits), inplace=True)
        

        if self.num_layers % 2 == 1:
            # iterate over layout permutations to recover measurements
            if self.property_set["virtual_permutation_layout"]:
                for cidx, qidx in self.property_set["virtual_permutation_layout"].get_physical_bits().items():
                    qaoa_circuit.measure(qidx, cidx)
            else:
                print("layout not found, assigining trivial layout")
                for idx in range(self.num_qubits):
                    qaoa_circuit.measure(idx, idx)
        else:
            for idx in range(self.num_qubits):
                qaoa_circuit.measure(idx, idx)
    
        return circuit_to_dag(qaoa_circuit)
        

Once again, let's check that this custom pass runs and builds a QAOA circuit with 3 layers:

In [ ]:
init = PassManager([QAOAPass(num_layers=3, num_qubits=10)])

In [ ]:
tmp_out = init.run(tmp)
tmp_out.count_ops()
tmp_out.draw('mpl', fold=-1)

### Complete the pipeline

Now that we can build our optimized QAOA ansatz, we need to fill out the remaining stages of the pipeline. We can use as a reference the output of the preset pass manager and just replace the init, pre-init and post-init stages with our custom pass managers. We will have to define a "post_init" step that takes care of the basis translation part to match the expected output in the default pipeline.

In [ ]:
from qiskit.transpiler import Layout

# We use the obtained path to define the initial layout
initial_layout = Layout.from_intlist(path, cost_layer.qregs[0])

In [ ]:
# The post init step unrolls the gates in the ansatz to the backend basis gates
post_init = PassManager(
    [
        UnrollCustomDefinitions(_sel, basis_gates=backend.operation_names, min_qubits=3),
        BasisTranslator(_sel, target_basis=backend.operation_names, min_qubits=3),
    ]
)

In [ ]:
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

staged_pm = generate_preset_pass_manager(3, backend, initial_layout=initial_layout)
staged_pm.pre_init = pre_init
staged_pm.init = init
staged_pm.post_init = post_init
# staged_pm.routing = None

### Run the StagedPassManager

In [ ]:
import time

t0_opt = time.time()
optimally_transpiled_qaoa = staged_pm.run(cost_layer)
t1_opt = time.time()

In [ ]:
# Print transpilation time
print(f"Transpilation time: {t1_opt - t0_opt:.2f} seconds")

circuit = optimally_transpiled_qaoa
# Output various metrics about the transpiled circuit
print(f'Count Ops: {circuit.count_ops()}')
print(f'Total circuit depth: {circuit.depth()}')
print(f'Number of 2-qubit gates: {circuit.size(lambda x: x.operation.num_qubits == 2)}')

# Output the score and depth specifically for 2-qubit gates, in color
print(Fore.MAGENTA + Style.BRIGHT + f'2-qubit gate depth: {circuit.depth(lambda x: x.operation.num_qubits == 2)}')
print(Fore.GREEN + f'Score: {score_func(circuit)}')

## Exercise 6: Grand Finale
<a id="exercise6"></a>
<div class="alert alert-block alert-success">
<b>Exercise 6: Advanced Circuit Optimization</b>

Let's use all the capabilities of the Qiskit SDK's Transpiler and Transpiler Service to achieve the best results. This exercise will test your skills to use the deepest customization techniques offered by Qiskit SDK to get to a better score for optimizing the provided QAOA Cost function circuit. 

**Challenge:**
1. Create and use a Custom Transpiler Pass within a StagedPassManager
2. The score as assessed by the grader needs to be less than **170** for the `cost_layer` circuit provided



**Some helpful documentation links for this exercise:**
- [Create a custom PassManager](https://docs.quantum.ibm.com/guides/dynamical-decoupling-pass-manager)
- [Write your own custom transpiler pass](https://docs.quantum.ibm.com/guides/custom-transpiler-pass)
- [Installing and using Transpiler plugins](https://docs.quantum.ibm.com/guides/transpiler-plugins)
- [Create a transpiler plugin](https://docs.quantum.ibm.com/guides/create-transpiler-plugin)

</div> 


<a id="scoring_func"></a>
<div class="alert alert-block alert-info">
    
⚠️ **About the scoring function** 

The scoring function evaluates your transpiled circuit based on its depth and gate usage, focusing on optimizing two-qubit gates, which are more resource-intensive on quantum hardware. It first checks if the circuit only uses the backend's basis gates and conforms to its qubit connectivity. If either check fails, a high penalty score is applied. Otherwise, the score is calculated by summing the depth of two-qubit gates and a reduced weight of single-qubit gates as follows:

$$score = n_{\text{2Q gate depth}} + n_{\text{1Q gate depth}}/10$$

**The goal is to minimize the score. Lower scores indicate more efficient circuits and higher circuit fidelity, so strive for optimization!** </div>

**Debugging tip:** When customizing transpilation settings, it can be extremely helpful to print out the list of passes that will be applied. This allows you to verify that your custom passes are being included in the correct order and that no unwanted default passes are being added. You can do this by adding a callback function as below:

In [ ]:
def callback_func(pass_, dag, time, property_set, count):
    print(f"PASS {count}: {pass_.name()} in {time}")
    print(f"ops: {dag.count_ops()}")
    print(f"dep: {dag.depth()}")
    print(f"========")

Examining the pass list helps you understand exactly how your circuit is being transpiled and can be invaluable for troubleshooting unexpected results or optimizing the transpilation process. If you notice passes that shouldn't be there or passes missing that you expected, you can adjust your [`generate_preset_pass_manager`](https://docs.quantum.ibm.com/api/qiskit/0.42/qiskit.transpiler.preset_passmanagers.generate_preset_pass_manager) configuration accordingly.

Experiment with modifying parameters associated with the transpilation stages to observe their impact on your circuit metrics:

In [ ]:
# Setup problem
from qiskit import QuantumCircuit
from networkx import barabasi_albert_graph, draw
graph = barabasi_albert_graph(n=50, m=6, seed=42)

# Build operators
from utils import build_max_cut_paulis
from qiskit.quantum_info import SparsePauliOp

local_correlators = build_max_cut_paulis(graph)
cost_operator = SparsePauliOp.from_list(local_correlators)
print(cost_operator)

# Define layers
num_qubits = cost_operator.num_qubits

# Build cost_layer
# We are taking advantage of the QAOAAnsatz class to build the cost layer, 
# note that we are giving it dummy initial state and mixer circuits.
from qiskit import QuantumCircuit
from qiskit.circuit.library import QAOAAnsatz
from qiskit.circuit import ParameterVector

dummy_initial_state = QuantumCircuit(num_qubits)  # the real initial state is defined later
dummy_mixer_operator = QuantumCircuit(num_qubits)  # the real mixer is defined later

cost_layer = QAOAAnsatz(
    cost_operator,
    reps=1,
    initial_state=dummy_initial_state,
    mixer_operator=dummy_mixer_operator,
    name="QAOA cost block",
)



In [ ]:
### Write your code below here ###

pm_ex6 = 


### Don't change any code past this line ###

circuit = pm_ex6.run(cost_layer,callback=callback_func)

# Output various metrics about the transpiled circuit
print(f'Count Ops: {circuit.count_ops()}')
print(f'Total circuit depth: {circuit.depth()}')
print(f'Number of 2-qubit gates: {circuit.size(lambda x: x.operation.num_qubits == 2)}')

# Output the score and depth specifically for 2-qubit gates, in color
print(Fore.MAGENTA + Style.BRIGHT + f'2-qubit gate depth: {circuit.depth(lambda x: x.operation.num_qubits == 2)}')
print(Fore.GREEN + f'Score: {score_func(circuit)}')

In [ ]:
# Run this cell to grade your answer
grade_ex6(pm_ex6, backend)

# Additional information

**Created by:** Elena Peña Tapia, Junye Huang, Vishal Sharathchandra Bajpe

**Advised by:** Abby Mitchel, Ali Javadi, Alexander Ivrii, Bryce Fuller, Matthew Trenish,  Shelly Garion

**Version:** 1.0.0